# Finding adversarial examples, in depth

In the quickstart, we used the default parameters for `find_adversarial_example`. Using the same example from the quickstart, we explore how to get more out of the function.

In [ ]:
using MIPVerify
using Gurobi
using JuMP
using Images

mnist = MIPVerify.read_datasets("MNIST")
n1params = MIPVerify.get_example_network_params("MNIST.n1")
sample_image = MIPVerify.get_image(mnist.train.images, 1);

function print_summary(d::Dict)
    # Helper function to print out output
    obj_val = getobjectivevalue(d[:Model])
    solve_time = getsolvetime(d[:Model])
    println("Objective Value: $(@sprintf("%.6f", obj_val)), Solve Time: $(@sprintf("%.2f", solve_time))")
end

function view_diff(diff::Array{<:Real, 2})
    n = 1001
    colormap("RdBu", n)[ceil.(Int, (diff+1)/2*n)]
end;

## `find_adversarial_example`

`find_adversarial_example` takes four positional arguments

```
find_adversarial_example(nnparams, input, target_selection, main_solver)
```

It also takes named arguments, each with the default value specified.

```
norm_order = 1
tolerance = 0
rebuild = false
invert_target_selection = false
pp = MIPVerify.AdditivePerturbationParameters()
tighten_bounds = true
model_build_solver: same as main solver, but with output suppressed and a time limit of 20s per solve.
```

We explore what each of these options allow us to do.

# Basic Options

## Specifying target categories for the adversarial example

`target_selection` and `invert_target_selection` control what the category we want the adversarial example to be classified in.

**Specification**: `target_selection` accepts either a single integer or a list of integers.

For example, if I wanted the original image (which is the digit 7) to be classified as the digit 0 or 8, I could run two separate solves with `target_selection=1` and `target_selection=9` (Julia is 1-indexed), finding closest adversarial examples at distance `15.0926` and `9.75700` ...

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 1, GurobiSolver(OutputFlag=0))
# OutputFlag=0 prevents any output from being printed out
print_summary(d)

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 9, GurobiSolver(OutputFlag=0));
print_summary(d)

Or I can can pass the targets as  `target_selection = [1, 9]`, where the same optimal value of `9.75700` is found.

Solve times for multiple target labels are typically on par with or faster than the aggregate solve times when solving with each target label in sequence.

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, [1, 9], GurobiSolver(OutputFlag=0));
print_summary(d)

A common use case is to have the adversarial example being in any category but the original:

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, [1, 2, 3, 4, 5, 6, 7, 9, 10], GurobiSolver(OutputFlag=0))
print_summary(d)

Rather than typing the full list of other categories, we can set `target_selection = 8`, and `invert_target_selection = true`.

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 8, GurobiSolver(OutputFlag=0), 
    invert_target_selection=true)
print_summary(d)

## More precise control over activations in output layer
### Default Behavior
The default option for `tolerance` is 0. This means that the activations in the softmax layer will take its maximum value for the target label (or at least one of them if more than one is specified), but that this maximum may not be unique.

In the example below, notice how the activations `y[8]` and `y[10]` (corresponding to labels 7 and 9) are equal, at 3.49171.

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0))
print_summary(d)
perturbed_sample_image = getvalue(d[:PerturbedInput])
y = perturbed_sample_image |> n1params

### Ensuring difference in activation between target labels and non-target labels

If we want to ensure that the maximum is unique, we can add a small tolerance. This is the minimum difference between the activation of the target label (or the maximum activation of any of the target labels) and any non-target label. In the example below, with a tolerance of 0.001, notice how `y[8] = 3.49243 = 0.00100 + 3.49143 = tolerance + y[10]`.

_This increases the objective value slightly (since the closest adversarial example with the required tolerance must be slightly further away.)_

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0), 
    tolerance = 0.001)
print_summary(d)
perturbed_sample_image = getvalue(d[:PerturbedInput])
y = perturbed_sample_image |> n1params

## Restricting the Family of Perturbations

### Additive Perturbations

The standard threat model is to allow each pixel to be perturbed independently, which is what happens by default:

In [ ]:
d = @time MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0));
perturbed_sample_image = getvalue(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

### Blurring Perturbations

We can restrict the perturbations to a blur with a 5x5 kernel instead. (We are still minimizing over the norm of the perturbation.)

In [ ]:
d = @time MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0),
    pp = MIPVerify.BlurPerturbationParameters((5, 5)));
perturbed_sample_image = getvalue(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

In [ ]:
diff = getvalue(d[:Perturbation])
view_diff(diff[1, :, :, 1])

## Minimizing Over Different Norms
### $l_1$
By default, we minimize the $l_1$ norm of the perturbation. This generally encourages sparsity in the perturbations. 

In this case, the minimum $l_1$ norm perturbation required for the image to be classified as a `9` is `3.171856.`

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0));
print_summary(d)
perturbed_sample_image = getvalue(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

We also show the difference between the perturbed image and the original image. Red is areas of decreased brightness and blue is areas of increased brightness.

In [ ]:
diff = getvalue(d[:Perturbation])
view_diff(diff[1, :, :, 1])

### $l_\infty$

We can also minimize over the $l_\infty$ norm. This generally results in large patches of the image being changed. 

In this case, the minimum $l_\infty$ norm perturbation required for the image to be classified as a `9` is `0.033522.`

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0),
    norm_order=Inf);
print_summary(d)
perturbed_sample_image = getvalue(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

In [ ]:
diff = getvalue(d[:Perturbation])
view_diff(diff[1, :, :, 1])

### $l_2$
With solvers that can handle MIQPs (like Gurobi), we can minimize over the $l_2$ norm. This generally takes a bit more time. 

In this case, the minimum $l_2$ norm perturbation required for the image to be classified as a `9` is `0.183 = sqrt(0.033522).`

In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0),
    norm_order=2);
print_summary(d)
perturbed_sample_image = getvalue(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

In [ ]:
diff = getvalue(d[:Perturbation])
view_diff(diff[1, :, :, 1])

# Advanced Options

## Models
Finding adversarial examples consists of two steps:

  1. Building a model that expresses the input-output constraints of the neural net, which involves determining upper and lower bounds on each intermediate value.
  2. Setting a target category (or multiple target categories) and solving to determine the minimal adversarial example that is classified in that category.

The first time we verify a neural net, we must do step 1, which can take a long time.

In [ ]:
d = @time MIPVerify.find_adversarial_example(n1params, sample_image, 1, GurobiSolver(OutputFlag=0), rebuild=true);
print_summary(d)

When re-running the solve (with the `rebuild` parameter defaulting to `false`), we load the model from cache and skip ahead straight to step 2; finding the adversarial example takes much less time.

In [ ]:
d = @time MIPVerify.find_adversarial_example(n1params, sample_image, 1, GurobiSolver(OutputFlag=0));
print_summary(d)

By setting `rebuild=true`, you force the MIP model to be rebuilt even if one exists; this is potentially useful if you want to spend more time expressing the neural net as an MIP model to speed up subsequent solves. (Think of this as an investment!)

### Removing Cached Models

If you find that your cached models are taking up too much space, you can remove them with `remove_cached_models()`.

## Advanced Solver-fu

We use two different solvers for the two steps of finding adversarial examples:

  1. the `model_build_solver`, used only to determine upper and lower bounds on intermediate values when expressing the input-output constraints in the neural net (i.e. if no cached model exists, or if `rebuild=true`), and 
  2. the `main_solver`, used to determine the minimal adversarial example for the loaded model.

### `tighten_bounds`

By default, we tighten the bounds on each intermediate value by solving an MIP using the `model_build_solver` (`tighten_bounds=true`. You can turn this off entirely and rely just on interval arithmetic. Notice how rebuilding the model is faster when we set `tighten_bounds=false`.

Bounds are tighter when `tighten_bounds=true` but the trade-off is that the process can take more time. (The solve itself takes about ~4s for the particular sample image and target label selected).

In [ ]:
MIPVerify.setloglevel!("notice")
@time MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0), rebuild=true);

In [ ]:
MIPVerify.setloglevel!("notice")
@time MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0), rebuild=true,
    tighten_bounds=false);

We can modify many of the parameters of the solver to change behavior:

We will be focusing on the parameters available via Gurobi (http://www.gurobi.com/documentation/7.5/refman/parameters.html), but other solvers often have similar options.

  
### `main_solver`

#### Muting Output
To mute the output from the `GurobiSolver`, set `OutputFlag=0`.


In [ ]:
d = MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver());
print_summary(d)

In [ ]:
MIPVerify.find_adversarial_example(n1params, sample_image, 10, GurobiSolver(OutputFlag=0));
print_summary(d)

#### Terminating early if a conditon is satisfied

Sometimes, finding an adversarial example takes a long time:

In [ ]:
MIPVerify.find_adversarial_example(n1params, sample_image, 2, GurobiSolver(),
    norm_order=Inf);

You may want to terminate early when a particular condition is satisfied. Common reasons are:

  1. Solve exceeding time limit
  2. Lower bound on robustness proved (i.e. `BestBd` increases above a pre-determined threshold)
  3. Counter-example found (i.e. `Incumbent` adversarial image found that is closer to the original image than expected).
  4. Difference between `Incumbent` and `BestBd` falls below a pre-determined threshold.
  
Fortunately, Gurobi has a parameter for all of these cases.

##### Terminate if time limit is reached
Set `TimeLimit`:

In [ ]:
MIPVerify.find_adversarial_example(n1params, sample_image, 2, GurobiSolver(TimeLimit=30),
    norm_order=Inf);

##### Terminate if lower bound on robustness proved

Set `BestBdStop` or `Cutoff`.

(`Cutoff` gives a different error message that is not currently processed correctly by the latest release of `Gurobi`).

In [ ]:
MIPVerify.find_adversarial_example(n1params, sample_image, 2, GurobiSolver(BestBdStop=0.05),
    norm_order=Inf);

##### Terminate if adversarial example found closer than expected robustness

Set `BestObjStop`.

In [ ]:
MIPVerify.find_adversarial_example(n1params, sample_image, 2, GurobiSolver(BestObjStop=0.2),
    norm_order=Inf);

##### Terminate if gap between `Incumbent` and `BestBd` is below threshold

Set `MIPGap`.

In [ ]:
MIPVerify.find_adversarial_example(n1params, sample_image, 2, GurobiSolver(MIPGap=0.4),
    norm_order=Inf);

### `model_build_solver`
The default model build solver has the same type as the `main_solver`, but uses the default settings for that solver type other than 1) muting the output and 2) setting a time limit of 20s per solve (i.e. per upper/lower bound per intermediate value).

The most common reason to pass in your own `model_build_solver` to modify the time limit per solve.

Whew! That was a lot. The next tutorial will introduce you to everything you can extract from the results dictionary.